In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

df=pd.read_excel("./Universalbank_1500-1.xlsx")

bank_df = df.drop('ZIP Code', axis=1)
bank_df.head(5)


,ID,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,4,1.0,2,0,0,0,0,0,1


In [2]:
def stats_mean(data):
    mean_vector=np.array([])
    for f in data.columns:
        m=data[f].mean()
        mean_vector=np.append(mean_vector, m)
        
    return mean_vector
stats_mean(bank_df)

array([7.50500000e+02, 4.52680000e+01, 2.00280000e+01, 7.42520000e+01,
       2.42266667e+00, 1.98320000e+00, 1.86266667e+00, 5.58200000e+01,
       1.01333333e-01, 1.06666667e-01, 6.06666667e-02, 6.05333333e-01,
       2.90000000e-01])

In [3]:
def within_scatter(data):
    
    data1 = data.drop('Personal Loan', axis=1)
    
    mean = stats_mean(data1)
    s=0
    for i in range(len(data1)):
        s+= np.outer(np.array(data1.iloc[i,:])-mean, (np.array(data1.iloc[i,:])-mean).transpose())
    return s


In [4]:
def btw_scatter(data_entire, data):
    
    data1 = data.drop('Personal Loan', axis=1)
    data1_entire = data_entire.drop('Personal Loan', axis=1)
    
    mean_total = stats_mean(data1_entire)
    mean_class = stats_mean(data1)
    
    n_class = len(data1)
    
    s = n_class*(np.outer((mean_class-mean_total),((mean_class-mean_total).transpose())))
    
    return s

In [5]:
def get_within_scatter(data, label):
    
    labels=data[label].unique()
    
    s=0
    for i in labels:
        
        df=data[data[label]==i]
        s += within_scatter(df)
    return s


In [6]:
def get_btw_scatter(data, label):
    
    labels=data[label].unique()
    
    s=0
    for i in labels:
        
        df=data[data[label]==i]
        s+=btw_scatter(data, df)
    return s


In [7]:
def sw_inv_sb(sw,sb):
    
    sw_inv = np.linalg.inv(sw)
    
    return sw_inv*sb


In [8]:
def eigens(M):
    
    eigenvalues, eigenvectors = np.linalg.eig(M)
    
    return eigenvalues, eigenvectors

eigenabc = eigens(sw_inv_sb(get_within_scatter(bank_df, "Personal Loan"),get_btw_scatter(bank_df, "Personal Loan")))


In [9]:
def desc_chose_eigen_vect(eigenvalues, eigenvectors):

    eigen_dict={}
    for i in range(len(eigenvalues)):
        eigen_dict[eigenvalues[i]]=eigenvectors[i]
    
    sorted_dict = dict(sorted(eigen_dict.items(), reverse=True))
    
    return sorted_dict

desc_chose_eigen_vect(eigenabc[0], eigenabc[1])


{0.6677390510719808: array([ 9.16053172e-04,  8.08134992e-04,  2.57231075e-03,  1.18693540e-04,
         2.95940007e-03,  6.48875206e-04, -1.91969302e-02, -6.60336553e-03,
         6.97510315e-03, -3.59151979e-02,  9.98096346e-01,  4.51187472e-02]),
 0.16207524089545433: array([-0.01554694,  0.06765116, -0.06287449,  0.62590181, -0.04498171,
         0.0869151 , -0.02700156,  0.0100891 ,  0.0224032 , -0.02059344,
         0.0334086 , -0.7661754 ]),
 0.14681060141205413: array([ 0.01716164, -0.08361842,  0.07666899, -0.74905358,  0.05708224,
        -0.11132071, -0.01530666,  0.00887498,  0.0188834 , -0.01734746,
         0.02774204, -0.63903298]),
 0.10763476179007919: array([-9.17971652e-01, -1.26213428e-01,  3.65071153e-01,  4.13408537e-02,
         1.24540774e-02, -7.65705287e-02,  1.98039674e-02, -1.74451430e-04,
         4.96566165e-04,  1.25636924e-04,  3.38125932e-04,  1.20444251e-03]),
 0.02156340346290305: array([-0.00899829, -0.00508268,  0.02392936,  0.00114616,  0.00254746,

In [10]:
def w_matrix(data, label, sorted_eigen):
    
    labels_count=data[label].nunique()
    
    keys= list(sorted_eigen.keys())
    w_matrix = np.array([])
    
    for i in range(labels_count-1):
        w_matrix = np.append(w_matrix, sorted_eigen[keys[i]])
    
    return w_matrix.transpose()
      
w_matrix(bank_df, "Personal Loan", desc_chose_eigen_vect(eigenabc[0], eigenabc[1]))

array([ 9.16053172e-04,  8.08134992e-04,  2.57231075e-03,  1.18693540e-04,
        2.95940007e-03,  6.48875206e-04, -1.91969302e-02, -6.60336553e-03,
        6.97510315e-03, -3.59151979e-02,  9.98096346e-01,  4.51187472e-02])

In [11]:
def projection(input_matrix, w_matrix):
    
    x = np.dot(input_matrix, w_matrix)
    
    return x

In [12]:
def LDA(data, label):
    
    input_1=data.drop(label, axis=1)
    
    input_matrix = input_1.values
    
    sw = get_within_scatter(data, label)
    sb = get_btw_scatter(data, label)
    m = sw_inv_sb(sw,sb)
    eigen = eigens(m)
    sorted_eigen = desc_chose_eigen_vect(eigen[0], eigen[1])
    w_matrix1 = w_matrix(data, label, sorted_eigen)
    x = projection(input_matrix, w_matrix1)
    
    return x

y=LDA(bank_df, "Personal Loan")

sorted_y = np.sort(y, kind='quicksort', order=None)[::-1]

print(sorted_y)

[ 2.54463791  2.52211258  2.50704797 ... -2.61078794 -2.89969819
 -3.67415745]
